In [1]:
import numpy as np 
import pandas as pd

In [3]:
filename = '~/NuFSGenMC_nominal.dat'
df = pd.read_csv(filename, delimiter=' ', names= ['pdg', 'Ereco', 'zreco', 'Etrue', 'ztrue', 'mcweight', 'flux_pion', 'flux_kaon'], skiprows=12)
df_small = df.sample(1000, random_state=0)
df_small.to_csv('./df_small.csv/', header=None, index=None)

In [ ]:
def build_gp(amplitude, length_scale, observation_noise_variance):
  """Defines the conditional dist. of GP outputs, given kernel parameters."""

  # Create the covariance kernel, which will be shared between the prior (which we
  # use for maximum likelihood training) and the posterior (which we use for
  # posterior predictive sampling)
  kernel = tfk.ExponentiatedQuadratic(amplitude, length_scale)

  # Create the GP prior distribution, which we will use to train the model
  # parameters.
  return tfd.GaussianProcess(
      kernel=kernel,
      index_points=X,
      observation_noise_variance=observation_noise_variance)

gp_joint_model = tfd.JointDistributionNamed({
    'amplitude': tfd.LogNormal(loc=0., scale=np.float64(1.)),
    'length_scale': tfd.LogNormal(loc=0., scale=np.float64(1.)),
    'observation_noise_variance': tfd.LogNormal(loc=0., scale=np.float64(1.)),
    'observations': build_gp,
})

In [ ]:
x = gp_joint_model.sample()
lp = gp_joint_model.log_prob(x)

print("sampled {}".format(x))
print("log_prob of sample: {}".format(lp))

In [ ]:
df.Ereco = np.round(df.Ereco,0)
df = df.groupby('Ereco').median().reset_index()
df['Ebin'] = pd.cut(df.Ereco, bins=500*10**np.linspace(0.0,1.3,14))
if len(df) > 5000:
    df_subsetted = df.sample(5000, random_state=0)
X = np.array(np.log(df_subsetted.Ereco)).reshape(-1,1)
y = np.log(df_subsetted.Etrue)
kernel2  = 1.0 * RBF() + WhiteKernel(noise_level=3)
gpr = GaussianProcessRegressor(kernel=kernel2,random_state=0).fit(X, y)